In [36]:
import os
import ast
import sys
sys.path.append('../')

import numpy as np
import pandas as pd

from rocket_rag.transform import *
# Import necessary libraries
from tsai.all import *
from sklearn.metrics import accuracy_score

In [37]:
cfg = json.load(open("../config/configs.json"))
ts_transform = TimeSeriesTransform(cfg)

In [38]:
INSTANCES_DIR = '../data/instances/'
INFERENCE_DIR = '../data/inference/'

STATES = ['normal', 
            'backlash1', 'backlash2',
            'lackLubrication1', 'lackLubrication2',
            'spalling1', 'spalling2', 'spalling3', 'spalling4', 'spalling5', 'spalling6', 'spalling7', 'spalling8']
LOADS= ['20kg', '40kg', '-40kg']

In [48]:
load = '20kg'
filenames = [os.listdir(os.path.join(INSTANCES_DIR, load, state)) for state in STATES]
filenames = [filename for sublist in filenames for filename in sublist]

X, y = [], []
for filename in filenames:
    load_num = load[:2]
    state = re.match(fr'(.*)_{load_num}', filename).group(1)
    df = pd.read_csv(os.path.join(INSTANCES_DIR, load, state, filename))
    tmp_cur = ts_transform.smoothing(ts_df=df, field='current')
    # tmp_pos = ts_transform.smoothing(ts_df=df, field='position_error')
    X.append(tmp_cur)
    y.append(state)

X = np.array(X)
y = np.array(y)

X, y, splits = combine_split_data([X], [y]) 

In [49]:
tfms = [None, [Categorize()]]
dls = get_ts_dls(X, y, splits=splits, tfms=tfms, bs=[64, 128])

In [56]:
test_filenames = [os.listdir(os.path.join(INFERENCE_DIR, load, state)) for state in STATES]
test_filenames = [filename for sublist in test_filenames for filename in sublist]

X_t, y_t = [], []
for filename in test_filenames:
    load_num = load[:2]
    state = re.match(fr'(.*)_{load_num}', filename).group(1)
    df = pd.read_csv(os.path.join(INFERENCE_DIR, load, state, filename))
    tmp_cur = ts_transform.smoothing(ts_df=df, field='current')
    # tmp_pos = ts_transform.smoothing(ts_df=df, field='position_error')
    X_t.append(tmp_cur)
    y_t.append(state)

X_t = np.array(X_t)
y_t = np.array(y_t)

X_t, y_t, splits = combine_split_data([X_t], [y_t])

In [ ]:
# Create and train the InceptionTime model
inceptiontime_accuracy_list = []
for i in range(1, 6):
    print(f"Model {i}")
    learn = ts_learner(dls, InceptionTime, metrics=accuracy, verbose=False)
    learn.fit_one_cycle(300)
    probas, targets, preds = learn.get_X_preds(X_t, y_t, with_decoded=True)
    # Convert the string to a list
    preditions = ast.literal_eval(preds)
    # Calculate accuracy or other metrics as needed
    inceptiontime_accuracy_list.append(sum(preditions == y_t) / len(y_t)*100)

In [116]:
np.mean(inceptiontime_accuracy_list)

86.15384615384616

In [ ]:
# Create and train the model
lstm_fcn_accuracy_list = []
for i in range(1, 6):
    print(f"Model {i}")
    learn = ts_learner(dls, LSTM_FCN, metrics=accuracy, verbose=False)
    learn.fit_one_cycle(300)
    probas, targets, preds = learn.get_X_preds(X_t, y_t, with_decoded=True)
    # Convert the string to a list
    preditions = ast.literal_eval(preds)
    # Calculate accuracy or other metrics as needed
    lstm_fcn_accuracy_list.append(sum(preditions == y_t) / len(y_t)*100)

In [66]:
np.mean(lstm_fcn_accuracy_list)

77.07692307692308

In [ ]:
# Create and train the model
tst_accuracy_list = []
for i in range(1, 6):
    print(f"Model {i}")
    learn = ts_learner(dls, TST, metrics=accuracy, verbose=False)
    learn.fit_one_cycle(300)
    probas, targets, preds = learn.get_X_preds(X_t, y_t, with_decoded=True)
    # Convert the string to a list
    preditions = ast.literal_eval(preds)
    # Calculate accuracy or other metrics as needed
    tst_accuracy_list.append(sum(preditions == y_t) / len(y_t)*100)

In [73]:
np.mean(tst_accuracy_list)

69.07692307692307

In [ ]:
# Create and train the model
lstm_accuracy_list = []
for i in range(1, 6):
    print(f"Model {i}")
    learn = ts_learner(dls, LSTM, metrics=accuracy, verbose=False)
    learn.fit_one_cycle(300)
    probas, targets, preds = learn.get_X_preds(X_t, y_t, with_decoded=True)
    # Convert the string to a list
    preditions = ast.literal_eval(preds)
    # Calculate accuracy or other metrics as needed
    lstm_accuracy_list.append(sum(preditions == y_t) / len(y_t)*100)

In [68]:
np.mean(lstm_accuracy_list)

19.230769230769234

In [ ]:
# Create and train the model
mwdn_accuracy_list = []
for i in range(1, 6):
    print(f"Model {i}")
    learn = ts_learner(dls, mWDN, metrics=accuracy, verbose=False)
    learn.fit_one_cycle(300)
    probas, targets, preds = learn.get_X_preds(X_t, y_t, with_decoded=True)
    # Convert the string to a list
    preditions = ast.literal_eval(preds)
    # Calculate accuracy or other metrics as needed
    mwdn_accuracy_list.append(sum(preditions == y_t) / len(y_t)*100)

In [71]:
np.mean(mwdn_accuracy_list)

73.84615384615384